<a href="https://colab.research.google.com/github/tmu-nlp/100knock2022/blob/main/koki/chapter10/chapter10.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab">
</a>

## 90. データの準備

機械翻訳のデータセットをダウンロードせよ．訓練データ，開発データ，評価データを整形し，必要に応じてトークン化などの前処理を行うこと．ただし，この段階ではトークンの単位として形態素（日本語）および単語（英語）を採用せよ．

<hr>

In [8]:
!tar zxvf kftt-data-1.0.tar.gz

kftt-data-1.0/
kftt-data-1.0/data/
kftt-data-1.0/data/orig/
kftt-data-1.0/data/orig/kyoto-tune.en
kftt-data-1.0/data/orig/kyoto-dev.ja
kftt-data-1.0/data/orig/kyoto-dev.en
kftt-data-1.0/data/orig/kyoto-train.en
kftt-data-1.0/data/orig/kyoto-tune.ja
kftt-data-1.0/data/orig/kyoto-train.ja
kftt-data-1.0/data/orig/kyoto-test.ja
kftt-data-1.0/data/orig/kyoto-test.en
kftt-data-1.0/data/tok/
kftt-data-1.0/data/tok/kyoto-tune.en
kftt-data-1.0/data/tok/kyoto-dev.ja
kftt-data-1.0/data/tok/kyoto-train.cln.en
kftt-data-1.0/data/tok/kyoto-dev.en
kftt-data-1.0/data/tok/kyoto-train.en
kftt-data-1.0/data/tok/kyoto-tune.ja
kftt-data-1.0/data/tok/kyoto-train.cln.ja
kftt-data-1.0/data/tok/kyoto-train.ja
kftt-data-1.0/data/tok/kyoto-test.ja
kftt-data-1.0/data/tok/kyoto-test.en
kftt-data-1.0/README.txt


In [9]:
!pip install ginza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
!cat kftt-data-1.0/data/orig/kyoto-train.ja | sed 's/\s+/ /g' | ginzame > train.ginza.ja
!cat kftt-data-1.0/data/orig/kyoto-dev.ja | sed 's/\s+/ /g' | ginzame > dev.ginza.ja
!cat kftt-data-1.0/data/orig/kyoto-test.ja | sed 's/\s+/ /g' | ginzame > test.ginza.ja

In [11]:
for src, dst in [
    ('train.ginza.ja', 'train.spacy.ja'),
    ('dev.ginza.ja', 'dev.spacy.ja'),
    ('test.ginza.ja', 'test.spacy.ja'),
]:
    #with open(src, encoding="utf-8_sig") as f:  # win
    with open(src) as f:  # unix
        lst = []
        tmp = []
        for x in f:
            x = x.strip()
            if x == 'EOS':
                lst.append(' '.join(tmp))
                tmp = []
            elif x != '':
                tmp.append(x.split('\t')[0])
    #with open(dst, 'w', encoding="utf-8_sig") as f:
    with open(dst, 'w') as f:
        for line in lst:
            print(line, file=f)

In [12]:
!python3 -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.9 MB 3.7 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.3.0
    Uninstalling en-core-web-sm-3.3.0:
      Successfully uninstalled en-core-web-sm-3.3.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [14]:
import re
import spacy

nlp = spacy.load("en_core_web_sm")
for src, dst in [
    ('kftt-data-1.0/data/orig/kyoto-train.en', 'train.spacy.en'),
    ('kftt-data-1.0/data/orig/kyoto-dev.en', 'dev.spacy.en'),
    ('kftt-data-1.0/data/orig/kyoto-test.en', 'test.spacy.en'),
]:
    with open(src) as f, open(dst, 'w') as g:
        for x in f:
            x = x.strip()
            x = re.sub(r'\s+', ' ', x)
            x = nlp.make_doc(x)
            x = ' '.join([doc.text for doc in x])
            print(x, file=g)

## 91. 機械翻訳モデルの訓練
90で準備したデータを用いて，ニューラル機械翻訳のモデルを学習せよ（ニューラルネットワークのモデルはTransformerやLSTMなど適当に選んでよい）．
<hr>

### 準備

fairseqのインストール

pip経由の場合旧バージョンをインストール可能、最新ビルドはgit cloneからのみ

```bash
git clone https://github.com/pytorch/fairseq
cd fairseq 
pip install --editable ./
```


In [15]:
!pip show fairseq

In [16]:
!pip install -q fairseq==0.10.2

     |████████████████████████████████| 1.7 MB 26.8 MB/s 
     |████████████████████████████████| 151 kB 13.8 MB/s 
     |████████████████████████████████| 92 kB 12.9 MB/s 
     |████████████████████████████████| 79 kB 3.0 MB/s 
     |████████████████████████████████| 117 kB 57.6 MB/s 
     |████████████████████████████████| 596 kB 66.2 MB/s 


sacremoses, sacrebleu という評価用のツールもインストール。 

BLEU という機械翻訳の評価スクリプトが使えるようになる



In [17]:
!pip install -q sacremoses
!pip install -q sacrebleu==1.5.1

     |████████████████████████████████| 880 kB 128 kB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 


### データの前処理

`fairseq-preprocess` というスクリプトでデータの前処理を行う

トークン化やサブワード化をして語彙サイズをコントロールしたりする

#### `fairseq-preprocess` オプション
最低限以下のコマンドを指定
- pref... 各ファイルのパス
- destdir...作成したデータセットの保存先
- task...翻訳、分類などモデルに学習させるタスクの種類

今回用いたコマンドは下記の通り
- -s または --source-lang...翻訳元の言語
- -t または --target-lang...翻訳後の言語
- thresholdsrc...map words appearing less than threshold times to unknown(Default: 0), 閾値以下の値の頻度で出現する単語を未知語にマップする
- thresholdtgt...map words appearing less than threshold times to unknown(Default: 0), 
- workers..number of parallel workers




In [18]:
!fairseq-preprocess -s ja -t en \
    --trainpref train.spacy \
    --validpref dev.spacy \
    --destdir data91  \
    --thresholdsrc 5 \
    --thresholdtgt 5 \
    --workers 20

2022-07-14 13:57:20 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_shard_count=1, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data91', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang='ja', srcdict=None, target_lang='en', task='translation', tensorboard_logdir=None, testpref=None, tgtdict=None, threshold_loss_scale=None, thresholdsrc=5, thresholdtgt=5, tokenizer=None, tpu=False, trainpref='train.spacy',

### 学習

`fairseq-train` というスクリプトで学習する

`%env CUDA_VISIBLE_DEVICES=0 `はGPU指定


#### `fairseq-train` オプション

- fp16...半精度浮動小数点による訓練(GPUスループットが向上する?わからん)
- arch ... 使用するモデルのアーキテクチャを指定(transformer)
- lr ... learing rate(default=0.25)
- max-epoch...そのままの意味、指定しないと無限に学習を続けるので注意
- optimizer...最適化(Possible choices: adadelta, adafactor, adagrad, adam, adamax, composite, cpu_adam, lamb, nag, sgd)


In [20]:
%env CUDA_VISIBLE_DEVICES=0 
!fairseq-train data91 \
    --fp16 \
    --save-dir save91 \
    --max-epoch 10 \
    --arch transformer --share-decoder-input-output-embed \
    --optimizer adam --clip-norm 1.0 \
    --lr 1e-3 --lr-scheduler inverse_sqrt --warmup-updates 2000 \
    --update-freq 1 \
    --dropout 0.2 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 8000 > 91.log

env: CUDA_VISIBLE_DEVICES=0
epoch 001: 100% 1822/1823 [10:44<00:00,  2.88it/s, loss=7.526, nll_loss=6.231, ppl=75.1, wps=18745.4, ups=2.85, wpb=6565.8, bsz=219.4, num_updates=1800, lr=0.0009, gnorm=0.868, clip=17, loss_scale=8, train_wall=35, wall=638]
epoch 001 | valid on 'valid' subset:   0% 0/7 [00:00<?, ?it/s]
epoch 001 | valid on 'valid' subset:  14% 1/7 [00:00<00:00,  6.42it/s]
epoch 001 | valid on 'valid' subset:  29% 2/7 [00:00<00:00,  7.89it/s]
epoch 001 | valid on 'valid' subset:  43% 3/7 [00:00<00:00,  8.50it/s]
epoch 001 | valid on 'valid' subset:  57% 4/7 [00:00<00:00,  8.87it/s]
epoch 001 | valid on 'valid' subset:  71% 5/7 [00:00<00:00,  9.16it/s]
epoch 001 | valid on 'valid' subset:  86% 6/7 [00:00<00:00,  8.74it/s]
epoch 001 | valid on 'valid' subset: 100% 7/7 [00:00<00:00,  8.79it/s]
epoch 002: 100% 1822/1823 [10:43<00:00,  2.77it/s, loss=6.691, nll_loss=5.273, ppl=38.67, wps=19136, ups=2.84, wpb=6729.1, bsz=253.8, num_updates=3600, lr=0.000745356, gnorm=0.622, clip=7

## 92. 機械翻訳モデルの適用
91で学習したニューラル機械翻訳モデルを用い，与えられた（任意の）日本語の文を英語に翻訳するプログラムを実装せよ．
<hr>

`fairseq-interactive`

学習済みモデルでテストデータを翻訳

- path ... 学習させたモデルを指定

91でepoch10まで学習したモデルでtest.spacy.ja(サブワード分割したテスト文)を翻訳

- サブワード分割

「道元（どうげん）は、鎌倉時代初期の禅僧。」

↓

「道元（どうげん） は 、 鎌倉 時代 初期 の 禅僧 。」



In [28]:
!fairseq-interactive --path save91/checkpoint10.pt data91 < test.spacy.ja | grep '^H' | cut -f3 > 92.out

/usr/local/lib/python3.7/dist-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/usr/local/lib/python3.7/dist-packages/fairseq/sequence_generator.py:651: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = idx // beam_size
Traceback (most recent call last):
  File

grep '^H'の目的？


#### 結果(上位5文)

##### 入力
1. 道元（どうげん） は 、 鎌倉 時代 初期 の 禅僧 。
2. 曹洞 宗 の 開祖 。
3. 晩年 に 希玄 と いう 異称 も 用い た 。
4. 同 宗旨 で は 高祖 と 尊称 さ れる 。
5. 諡 は 、 仏性 伝 東国 師 、 承 陽 大師 _ ( 僧 ) 。

##### 出力
1. Zen priests in the early Kamakura period .
2. Founder of Soto sect .
3. His name was also written as <unk> .
4. It is also referred to as <unk> .
5. His posthumous Buddhist name was <unk> .

##### 正解
1. Dogen was a Zen monk in the early Kamakura period .
2. The founder of Soto Zen
3. Later in his life he also went by the name Kigen .
4. Within the sect he is referred to by the honorary title Koso .
5. Posthumously named Bussho Dento Kokushi , or Joyo - Daishi .

## 93. BLEUスコアの計測
91で学習したニューラル機械翻訳モデルの品質を調べるため，評価データにおけるBLEUスコアを測定せよ．

<hr>

In [29]:
!fairseq-score --sys 92.out --ref test.spacy.en

Namespace(ignore_case=False, order=4, ref='test.spacy.en', sacrebleu=False, sentence_bleu=False, sys='92.out')
BLEU4 = 4.55, 19.3/6.3/2.7/1.3 (BP=1.000, ratio=1.845, syslen=26885, reflen=14569)


## 94. ビーム探索

91で学習したニューラル機械翻訳モデルで翻訳文をデコードする際に，ビーム探索を導入せよ．ビーム幅を1から100くらいまで適当に変化させながら，開発セット上のBLEUスコアの変化をプロットせよ．

<hr>

In [ ]:
!for N in `seq 1 10` ; do fairseq-interactive --path save91/checkpoint10.pt --beam $N data91 < test.spacy.ja | grep '^H' | cut -f3 > 94.$N.out  done

In [ ]:
!for N in `seq 1 10` ; do fairseq-score --sys 94.$N.out --ref test.spacy.en > 94.$N.score done

グラフ化

In [ ]:
import matplotlib.pyplot as plt

def read_score(filename):
    with open(filename) as f:
        x = f.readlines()[1]
        x = re.search(r'(?<=BLEU4 = )\d*\.\d*(?=,)', x)
        return float(x.group())

xs = range(1, 21)
ys = [read_score(f'94.{x}.score') for x in xs]
plt.plot(xs, ys)
plt.show()